# Lecture des fichiers

## Importation des librairies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from fastddm.fit import simple_structure_function, fit
from fastddm.azimuthalaverage import AAReader

## Définition de la vidéo à lire

In [ ]:
video = 25 
date="data/2025-02-25/"
size=512
filename = date+"extracted/resized_size"+str(size)+"_"+str(video)+".aa.ddm"

## Lecture

In [ ]:
r = AAReader(filename)
aa = r.load()
q = aa.k
time = aa.tau

In [ ]:
type(aa)

In [ ]:
q.shape

In [ ]:
time.shape

In [ ]:
aa.data.shape

## Définition des normes de vecteurs d'onde à afficher

In [ ]:
test_k = np.array([20, 30, 80, 90, 100], dtype=np.int64)

In [ ]:
q[test_k]

## Affichage

In [ ]:
for tk in tqdm(test_k):    
    plt.scatter(time, aa.data[tk]/np.max(aa.data[tk]), marker="o", s=3, label="$q={:.3f}\ [\mu m^{{-1}}]$".format(q[tk]))
plt.xscale('log')
plt.yscale('log')
plt.xlabel("$\Delta t\ [s]$")
plt.ylabel(r"$\frac{\langle|\Delta I(q,\Delta t)|^2 \rangle}{\max_{\Delta t}\langle|\Delta I(q,\Delta t)|^2 \rangle}$")
plt.title("Fonction de décorrélation normalisée (log-log)")
plt.legend()

In [ ]:
for tk in tqdm(test_k):    
    plt.scatter(time, aa.data[tk]/np.max(aa.data[tk]), marker="o", s=3, label="$q={:.3f}\ [\mu m^{{-1}}]$".format(q[tk]))
plt.xlabel("$\Delta t\ [s]$")
plt.ylabel(r"$\frac{\langle|\Delta I(q,\Delta t)|^2 \rangle}{\max_{\Delta t}\langle|\Delta I(q,\Delta t)|^2 \rangle}$")
plt.title("Fonction de décorrélation (lin-lin)")
plt.legend()

## Fit de quelques valeurs

In [ ]:
simple_structure_function.set_param_hint("B", min=-np.inf, max=np.inf, value=0.0)
fit_time = np.logspace(-2, 0.7)
labelstr = r"$A = {A:.2f},\ B = {B:.2f},\ \tau = {tau:.2f}s$"
weights = 1/np.sqrt(aa.tau)

In [ ]:
fig = plt.figure(figsize=(9, 6))
val = {}
for tk in tqdm(test_k):
    # fit for the given fixed index/k
    result = fit(simple_structure_function, xdata=aa.tau, ydata=aa.data[tk]/np.max(aa.data[tk]), weights=weights)
    val[tk]=result.best_values
    
    # plot original data
    plt.scatter(aa.tau, aa.data[tk]/np.max(aa.data[tk]), marker="o", s=6, 
                label="$q={:.3f}\ [\mu m^{{-1}}]$".format(q[tk]))
    
    # and the fit
    plt.plot(fit_time, simple_structure_function.eval(**result.best_values, dt=fit_time), linestyle="--", linewidth=0.8, label=labelstr.format(**result.best_values))

plt.xlabel("$\Delta t\ [s]$")
plt.ylabel("Fonction de décorrélation normalisée")
plt.title("Fonction de décorrélation normalisée fittée (lin-lin)")
plt.legend()

In [ ]:
fig = plt.figure(figsize=(9, 6))
for tk in tqdm(test_k):
    plt.scatter(aa.tau, aa.data[tk]/np.max(aa.data[tk]), marker="o", s=6, 
                label="$q={:.3f}\ [\mu m^{{-1}}]$".format(q[tk]))

    plt.plot(fit_time, simple_structure_function.eval(**val[tk], dt=fit_time), linestyle="--", linewidth=0.8, label=labelstr.format(**val[tk]))
plt.xscale("log")
plt.yscale("log")
plt.xlabel("$\Delta t\ [s]$")
plt.ylabel("Fonction de décorrélation normalisée")
plt.title("Fonction de décorrélation normalisée fittée (log-log)")
plt.legend()

## Résidus

In [ ]:
for tk in tqdm(test_k):    
    # plot original data
    plt.scatter(aa.tau, aa.data[tk]/np.max(aa.data[tk])-simple_structure_function.eval(**val[tk], dt=aa.tau), marker="o", s=6, 
                label="$q={:.3f}\ [\mu m^{{-1}}]$".format(q[tk]))
   
plt.xlabel("$\Delta t\ [s]$")
plt.ylabel("Fonction de décorrélation (résidus)")
plt.title("Fonction de décorrélation fittée (résidus)")
plt.legend()

## Fit complet

In [ ]:
q_index = [30,100]

In [ ]:
tau = []
for i in tqdm(range(*q_index)):
    result = fit(simple_structure_function, xdata=aa.tau, ydata=aa.data[i]/np.max(aa.data[i]), weights=weights)
    tau.append(result.best_values["tau"])

In [ ]:
x = q[q_index[0]:q_index[1]]
y = tau
x_th = np.linspace(np.min(x), np.max(x), 100)

In [ ]:
x.shape

In [ ]:
len(y)

In [ ]:
y=np.array(y)
x=np.array(x)
x = x[y>1e-3]
y=y[y>1e-3]


In [ ]:
plt.xscale("log")
plt.yscale("log")
plt.scatter(x, y, marker='+', label="Données", color='r')
plt.xlabel(r"$q (\mu m)^{-1}$")
plt.ylabel(r"$\tau (s)$")
a, b = np.polyfit(np.log(x), np.log(y), 1)
plt.plot(x_th, np.exp(a*np.log(x_th) + b), label=f"Modèle $ax+b$ a = {a:.2f} et b = {b:.2f}")
plt.legend()

## Résidus

# Coefficient de diffusion

In [ ]:
D = np.exp(-b)*1e-12 # en m2/s
print(D)

In [ ]:
T = 273.15 - 22

In [ ]:
a = 345e-9/2

In [ ]:
import scipy.constants as c

In [ ]:
eta = c.Boltzmann*T/(6*np.pi*D*a)
print(eta)
